Take a look at the [repository](https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix) for more information

# Install repo and load dataset

In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

In [ ]:
import os
os.chdir('content/pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

In [ ]:
os.chdir("datasets")
!pip install -q opendatasets

import opendatasets as od
import pandas as pd

kaggle_id = {"username":"your_username_here","key":"your_kaggle_key_here"}
od.download('https://www.kaggle.com/datasets/ashishjangra27/gender-recognition-200k-images-celeba')

In [ ]:
os.rename("gender-recognition-200k-images-celeba", "male2female")
os.rename("male2female/Dataset/Test/Male", "male2female/testA")
os.rename("male2female/Dataset/Test/Female", "male2female/testB")
os.rename("male2female/Dataset/Train/Male", "male2female/trainA")
os.rename("male2female/Dataset/Train/Female", "male2female/trainB")

In [ ]:
import shutil
shutil.rmtree("male2female/Dataset")

In [ ]:
os.chdir("../")

# Training

-   `python train.py --dataroot ./datasets/horse2zebra --name horse2zebra --model cycle_gan`

Change the `--dataroot` and `--name` to your own dataset's path and model's name. Use `--gpu_ids 0,1,..` to train on multiple GPUs and `--batch_size` to change the batch size. I've found that a batch size of 16 fits onto 4 V100s and can finish training an epoch in ~90s.

Once your model has trained, copy over the last checkpoint to a format that the testing model can automatically detect:

Use `cp ./checkpoints/horse2zebra/latest_net_G_A.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class A to class B and `cp ./checkpoints/horse2zebra/latest_net_G_B.pth ./checkpoints/horse2zebra/latest_net_G.pth` if you want to transform images from class B to class A.


In [ ]:
!python train.py --dataroot ./datasets/male2female/ --name male2female --model cycle_gan --display_id -1 --batch_size 16 --num_threads 0 --gpu_ids 0 --netG resnet_6blocks --ngf 16 --ndf 16 --max_dataset_size 10000 --n_epochs 100

# Testing

-   `python test.py --dataroot datasets/horse2zebra/testA --name horse2zebra_pretrained --model test --no_dropout`

Change the `--dataroot` and `--name` to be consistent with your trained model's configuration.

> from https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix:
> The option --model test is used for generating results of CycleGAN only for one side. This option will automatically set --dataset_mode single, which only loads the images from one set. On the contrary, using --model cycle_gan requires loading and generating results in both directions, which is sometimes unnecessary. The results will be saved at ./results/. Use --results_dir {directory_path_to_save_result} to specify the results directory.

> For your own experiments, you might want to specify --netG, --norm, --no_dropout to match the generator architecture of the trained model.

In [ ]:
!python test.py --dataroot datasets/male2female --name male2female --model cycle_gan --netG resnet_6blocks --ngf 16 --ndf 16 --gpu_ids 0

# Visualize

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/male2female/test_latest/images/160002_real_A.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/male2female/test_latest/images/160002_fake_B.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/male2female/test_latest/images/160002_rec_A.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/male2female/test_latest/images/160002_real_B.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/male2female/test_latest/images/160002_fake_A.png')
plt.imshow(img)

In [ ]:
import matplotlib.pyplot as plt

img = plt.imread('./results/male2female/test_latest/images/160002_rec_B.png')
plt.imshow(img)

# Save and download trained model & results

In [ ]:
from google.colab import files

os.chdir("../../")

!zip -r /content/cycleGAN_model_results.zip /content/pytorch-CycleGAN-and-pix2pix/checkpoints/male2female
!zip -r /content/cycleGAN_photo_results.zip /content/pytorch-CycleGAN-and-pix2pix/results/male2female

files.download("/content/cycleGAN_model_results.zip")
files.download("/content/cycleGAN_photo_results.zip")